利用tensorflow简单实现一个IOU

In [19]:
import tensorflow as tf

In [22]:
def calc_iou(box1,box2):
    left_up = tf.maximum(box1[:2 ],box2[:2])
    right_down = tf.minimum(box2[2: ],box2[2:])
    intersection = tf.maximum(0.,right_down - left_up)
    inter_square = intersection[0] * intersection[1]
    
    box1_square = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_square = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    iou = inter_square / (box1_square + box2_square - inter_square)
    
    return iou

In [23]:
with tf.Session() as sess:
    iou = calc_iou([10.,20.,60.,60.],[50.,40.,80.,90.])
    print(sess.run(iou))
    iou = calc_iou([10.,20.,40.,60.],[10.,40.,80.,90.])
    print(sess.run(iou))

0.75
2.9166667


进行NMS的算法

In [24]:
def nms(boxes,threshold=0.7):
    boxes.sort(key=lambda boxes: boxes[4],reverse=True)
    res_boxes = []
    while(len(boxes) != 0):
        res_boxes.append(boxes[0])
        del boxes[0]
        
        keep_num = 0
        for i in range(len(boxes)):
            iou = calc_iou(res_boxes[-1][0:4],boxes[keep_num][0:4])
            if iou > threshold:
                del boxes[keep_num]
            else:
                    keep_num += 1
                    
        return res_boxes

In [26]:
# boxes = [[0,0,50,50,0.9],
#          [0,0,60,50,0.85],
#          [40,40,70,70,0.4],
#          [35,45,70,80,0.7]]

# res_boxes = nms(boxes,1)
# print(res_boxes)

tensorflow 实现代价函数

In [29]:
outputs = tf.random_uniform([2,8],0,1, dtype=tf.float32)
labels = tf.constant([[0,0,0,0,0,0,0,0],
                     [1,0.2,0.3,0.3,0.8,0,0,1]],dtype=tf.float32)


In [32]:
labels_confidence = labels[:,0:1]
mask = tf.concat([tf.ones_like(labels)[:,0:1],
                (tf.ones_like(labels) * labels_confidence)[:,1: ]],1)
outputs = outputs *mask
cost = tf.reduce_mean(tf.square(labels - outputs))